In [1]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Embedding Gecko (Multilingual)

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [4]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [5]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession,
)

from vertexai.preview.generative_models import grounding
from vertexai.preview.language_models import TextEmbeddingModel

### Initalize Vertex AI

In [6]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function

In [7]:
import numpy as np
import torch
import torch.nn as nn
from typing import List, Optional

#------------------------------------------------------------
def gecko_embeddings(sentences: List[str],model: TextEmbeddingModel) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception as e:
        print(f"Exception : {str(e)}")
        return [None for _ in range(len(sentences))]

#------------------------------------------------------------
def dot_product(a, b):
  """ torch dot product similarity """

  return torch.dot(a, b)

#------------------------------------------------------------
def cosine_similarity(a, b):
  """ torch cosine similarity """

  cos = nn.CosineSimilarity(dim=0, eps=1e-6)
  output = cos(a, b)
  return output

#------------------------------------------------------------
def matrix_multiplication(a, b):
  """ torch matrix multiplication """

  if len(a.shape) == 1: a = a.unsqueeze(0)
  if len(b.shape) == 1: b = b.unsqueeze(0)

  a_norm = a / a.norm(dim=1)[:, None]
  b_norm = b / b.norm(dim=1)[:, None]
  return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

#------------------------------------------------------------
def print_similarity(input_embed, target_embed):
  import torch

  score0 = dot_product(torch.Tensor(input_embed[0]),torch.Tensor(input_embed[0]) )
  score1 = dot_product(torch.Tensor(input_embed[0]),torch.Tensor(target_embed[0]) )
  score2 = dot_product(torch.Tensor(input_embed[0]),torch.Tensor(target_embed[1]) )
  score3 = dot_product(torch.Tensor(input_embed[0]),torch.Tensor(target_embed[2]) )
  score4 = dot_product(torch.Tensor(input_embed[0]),torch.Tensor(target_embed[3]) )

  print(f"score0[{score0}]")
  print(f"score1[{score1}]")
  print(f"score2[{score2}]")
  print(f"score3[{score3}]")
  print(f"score4[{score4}]")


### Test sentences

In [8]:
sentence0 = """ 2024년에는 인공지능 기술을 활용한 다양한 고객서비스에 대한 요구를 많이 한다. """

sentence1 = """ 2024년에는 경기 둔화 여파로 사람들은 절약을 할 것이고 인공지능 기술을 활용한 다양한 고객서비스에 대한 요구를 많이 한다.
인공지능을 활용하기 위한 고객 서비스는 다양한 형태의 활용사례를 만들수 있으며, 금융회사의 고객 상담 센터가 가장 적합한 형태의 서비스가 될 가능성이 있다.
기타 다양한 산업 분야에서의 지식검색을 통한 고객서비스도 좋은 사례가 될수 있다.
"""

sentence2 = """ 2024년 금융소비 트렌드와 금융 기회를 보면 2024년에는 경기 둔화의 여파로 안전하고 절약 지향적인 재무관리 태도가 소비자의 금융생활 전반에 확산될 전망이다.
또한, 인공지능(AI) 등 디지털 혁신이 고도화되면서 1:1 맞춤형 자산관리나 웨어러블(Wearable) 기반의 간편 결제 서비스에 대한 기대는 점점 커지고 있다.
사회적 의식이 성숙하면서 금융권의 책임경영이 중시되는 경향과 연령이 낮아지는 미래 손님군의 특징도 눈여겨 볼만하다.
"""

sentence3 = """ 피아노는 17세기 말, 이탈리아의 피렌체에서 처음으로 그 모습을 드러냈다.
피아노는 제작자 ‘바르톨로메오 크리스토포리(Bartolomeo Cristofori, 1655-1732)’가 처음 발명한 것으로 추측되며,
당시 크리스토포리는 피렌체를 대표하는 가문 ‘메디치’를 위해 건반악기를 제작하고 있었다. 피아노의 전신 하프시코드, 스피넷등을 제작하던 크리스토포리는 1689년경 피아노 제작에 착수했을 것으로 추정되며,
1710년경에는 피아노 3대를 완성했다는 기록이 남아있다.
"""

sentence4 = """ 2024년에는 사람들은 점점 소비를 줄이게 될것이다. """

### Execution

In [9]:
import torch

gecko_model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")

input_sentences = [sentence0]
target_sentences = [sentence1, sentence2, sentence3, sentence4]

input_embeddings = gecko_embeddings(sentences=input_sentences, model = gecko_model )
target_embeddings = gecko_embeddings(sentences=target_sentences, model = gecko_model )

print_similarity(input_embeddings, target_embeddings )


score0[0.9999979734420776]
score1[0.8937548995018005]
score2[0.822515070438385]
score3[0.48196762800216675]
score4[0.7435875535011292]
